In [1]:
from qiskit import *
from numpy import *
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_vector, array_to_latex
from qiskit.quantum_info import state_fidelity
from qiskit.quantum_info import *

import matplotlib.pyplot as plt
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error
from qiskit.providers.aer.noise import NoiseModel
import qiskit.circuit.library as qulib
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel, mixed_unitary_error
import matplotlib.pyplot as plt
import numpy as np


aer_sim = AerSimulator()
#aer_sim = Aer.get_backend('statevector_simulator')

In [ ]:
def eaqecc(p):
    qm= QuantumRegister(1,'q_main')
    qa= QuantumRegister(2,'q_ancilla')
    qe=QuantumRegister(2,'q_entangled')
    qs = QuantumRegister(4,'q_syndrome')
    creg= ClassicalRegister(4,'syndrome_bits')
    res= ClassicalRegister(1,"result")
    qc= QuantumCircuit(qs,qe,qa,qm,creg)
    qc.h(qe[0])
    qc.cx(qe[0],qe[1])



    initial_state = [1,0]   # Define state |q_0>
    qc.initialize(initial_state, qm)
    dm1= DensityMatrix(initial_state)
    ##encoding circuit
    qc.h(qa[0])
    qc.h(qa[1])
    qc.cx(qa[1],qm)
    qc.cx(qa[0],qa[1])
    qc.h(qm)
    qc.s(qa[0])
    qc.h(qa[1])
    qc.cx(qa[0],qa[1])
    qc.cx(qa[0],qm)
    qc.h(qa[0])
    qc.cx(qe[1],qa[0])
    qc.cx(qe[1],qm)
    qc.h(qe[1])
    qc.h(qm)
    qc.cx(qe[1],qa[0])
    qc.cx(qe[1],qa[1])
    qc.h(qa[0])
    qc.h(qa[1])
    qc.swap(qe[1],qa[0])
    # cf = Clifford(qc)
    # print(cf.to_labels(mode="S"))
    # Channel starts here
    qc.barrier()

    X= qi.Operator.from_label('X')
    Y= qi.Operator.from_label('Y')
    Z= qi.Operator.from_label('Z')
    I= qi.Operator.from_label('I')


    noise_ops = [sqrt(1-3*p/4)*I,sqrt(p/4)*X,sqrt(p/4)*Y,sqrt(p/4)*Z]


    channel = Kraus(noise_ops)
    qc.append(channel,[5])
    qc.append(channel,[6])
    qc.append(channel,[7])
    qc.append(channel,[8])



    qc.barrier()
    # channel ends here

    qc.h(qs)
    qc.barrier()
    # ZIZZX
    qc.cx(qs[0],qe[0])
    qc.cz(qs[0],qe[1])
    qc.cz(qs[0],qa[0])
    qc.cz(qs[0],qm)

    #IZXZZ
    qc.cz(qs[1],qe[0])
    qc.cz(qs[1],qe[1])
    qc.cx(qs[1],qa[0])
    qc.cz(qs[1],qa[1])

    #XYYZI
    qc.cz(qs[2],qe[1])
    qc.cy(qs[2],qa[0])
    qc.cy(qs[2],qa[1])
    qc.cx(qs[2],qm)


    #-ZXXYI
    qc.cy(qs[3],qe[1])
    qc.cx(qs[3],qa[0])
    qc.cx(qs[3],qa[1])
    qc.cz(qs[3],qm)


    qc.barrier()
    qc.h(qs)
    qc.measure(qs,creg)
    #display(qc.draw())





    # print(counts)
    qc.barrier()
    # error correction circuit
    qc.x(qm).c_if(creg,1)
    qc.y(qm).c_if(creg,5)
    qc.z(qm).c_if(creg,12)

    qc.x(qa[1]).c_if(creg,14)
    qc.y(qa[1]).c_if(creg,2)
    qc.z(qa[1]).c_if(creg,4)

    qc.x(qa[0]).c_if(creg,13)
    qc.y(qa[0]).c_if(creg,3)
    qc.z(qa[0]).c_if(creg,6)

    qc.x(qe[1]).c_if(creg,7)
    qc.y(qe[1]).c_if(creg,15)
    qc.z(qe[1]).c_if(creg,0)

    # decoding circuit
    qc.barrier()
    qc.swap(qe[1],qa[0])
    qc.h(qa[1])
    qc.h(qa[0])
    qc.cx(qe[1],qa[1])
    qc.cx(qe[1],qa[0])
    qc.h(qm)
    qc.h(qe[1])
    qc.cx(qe[1],qm)
    qc.cx(qe[1],qa[0])
    qc.h(qa[0])
    qc.cx(qa[0],qm)
    qc.cx(qa[0],qa[1])
    qc.h(qa[1])
    qc.s(qa[0])
    qc.h(qm)
    qc.cx(qa[0],qa[1])
    qc.cx(qa[1],qm)
    qc.h(qa[1])
    qc.h(qa[0])
    # qc.measure(qm,res)

    # backend = BasicAer.get_backend("qasm_simulator")
    # tqc = transpile(qc, backend)
    # job = backend.run(tqc, shots=1000)
    # result = job.result()
    # counts = result.get_counts(tqc)
    # plot_histogram(counts)

    qc.save_density_matrix(qubits=qm)

    sim_density = AerSimulator()
    job= sim_density.run(qc)
    result = job.result().data()
    dm_after_channel= result.get('density_matrix')


    return state_fidelity(dm1,dm_after_channel)

In [ ]:
p_values = np.arange(0, 1.01,0.01 )

px = linspace(0, 1, 100)

# Calculating fidelity from theoretical formula

x= 1 - 27*p**2/8 + 15*p**3/4 -9*p**4/8  


fidelities_eafive = []

fidelities_actual = []

# Loop through p values, calculate fidelities, and store results
for p in p_values:
    fidelities_eafive.append(eaqecc(p))

# Create the plot
plt.figure(figsize=(8, 6))
plt.plot(p_values,fidelities_eafive, marker='o',linewidth=3, linestyle='-',label='Experimental fidelity of EA Five qubit Code')
plt.plot(px,fidelities_actual,linewidth=3, linestyle='-',label='Theoretical fidelity of EA Five qubit Code')
#plt.plot(p,x,linewidth=1,label='Actual fidelity by formula')

plt.xlabel('Depolarizing Error Probability (p)')
plt.ylabel('State Fidelity')
plt.title('State Fidelity vs. Depolarizing Error in Quantum Circuit')
plt.legend()
plt.grid(True)
plt.show()